In [3]:
import json
import pprint as pp
import requests as r
from tapipy.tapis import Tapis

# Set url
url = "https://tacc.tapis.io"

# Create Tapis object
t = Tapis(base_url = url,
          username = 'username',
          password = 'password')
t.get_tokens()

# V3 Headers
header_dat = {"X-Tapis-Token": t.access_token.access_token}

### PgREST Endpoint Calls: Get, Post, Delete for /manage/tables and /data/{table_id}

In [13]:
# Get manage tables
res = r.get(f'{url}/v3/pgrest/manage/tables',
             headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': [{'comments': '',
             'endpoints': ['GET_ONE', 'GET_ALL', 'CREATE', 'UPDATE', 'DELETE'],
             'primary_key': 'initial_table_id',
             'root_url': 'init',
             'table_id': 1,
             'table_name': 'initial_table',
             'tenant': 'dev',
             'tenant_id': 'dev'}],
 'status': 'success',
 'version': 'dev'}


In [69]:
# Create a test table definition (init_table_1) that we're going to make a table out of.
init_table_1 = {'table_name': 'initial_table',
                'root_url': 'init',
                'columns': {'col_one': {'null': True, 'data_type': 'varchar', 'char_len': 255},
                            'col_two': {'null': True, 'data_type': 'integer'},
                            'col_three': {'null': True, 'data_type': 'integer'},
                            'col_four': {'null': False, 'data_type': 'boolean', 'default': True},
                            'col_five': {'null': True, 'data_type': 'varchar', 'char_len': 255}}}

In [74]:
# Create table
res = r.post(f'{url}/v3/pgrest/manage/tables',
             data=json.dumps(init_table_1),
             headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': {'comments': '',
            'endpoints': ['GET_ONE', 'GET_ALL', 'CREATE', 'UPDATE', 'DELETE'],
            'root_url': 'init',
            'table_id': 421,
            'table_name': 'initial_table'},
 'status': 'success',
 'version': 'dev'}


In [75]:
# Add row to table
data = {"col_two": 32, "col_three": 90, "col_four": False}
res = r.post(f'{url}/v3/pgrest/data/init',
            data = json.dumps({"data": data}),
            headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': [{'_pkid': 1,
             'col_five': None,
             'col_four': False,
             'col_one': None,
             'col_three': 90,
             'col_two': 32,
             'initial_table_id': 1}],
 'status': 'success',
 'version': 'dev'}


In [76]:
# Get table rows
res = r.get(f'{url}/v3/pgrest/data/init',
            headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': [{'_pkid': 1,
             'col_five': None,
             'col_four': False,
             'col_one': None,
             'col_three': 90,
             'col_two': 32,
             'initial_table_id': 1}],
 'status': 'success',
 'version': 'dev'}


In [78]:
# Add row to table
data = {"col_one": "hello", "col_two": 22, "col_three": 90, "col_four": False, "col_five": "hehe"}
res = r.post(f'{url}/v3/pgrest/data/init',
             data = json.dumps({"data": data}),
             headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': [{'_pkid': 2,
             'col_five': 'hehe',
             'col_four': False,
             'col_one': 'hello',
             'col_three': 90,
             'col_two': 22,
             'initial_table_id': 2}],
 'status': 'success',
 'version': 'dev'}


## Views - Creating a view for the table just created

In [80]:
# Create view
res = r.post(f'{url}/v3/pgrest/manage/views',
             data=json.dumps({"view_name": "testview",
                              "select_query": "*",
                              "from_table": "initial_table",
                              "comments": "Test for new view.",
                              "permission_rules": ["PGREST_ADMIN"]}),
             headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': {'comments': 'Test for new view.',
            'endpoints': ['GET_ONE', 'GET_ALL', 'CREATE', 'UPDATE', 'DELETE'],
            'root_url': 'testview',
            'view_id': 2,
            'view_name': 'testview'},
 'status': 'success',
 'version': 'dev'}


In [81]:
# Get manage view
res = r.get(f'{url}/v3/pgrest/manage/views',
                headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': [{'comments': 'Test for new view.',
             'endpoints': ['GET_ONE', 'GET_ALL', 'CREATE', 'UPDATE', 'DELETE'],
             'manage_view_id': 2,
             'root_url': 'testview',
             'tenant_id': 'tacc',
             'view_name': 'testview'}],
 'status': 'success',
 'version': 'dev'}


In [83]:
# Get view
res = r.get(f'{url}/v3/pgrest/views/testview',
            headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': [{'col_five': None,
             'col_four': False,
             'col_one': None,
             'col_three': 90,
             'col_two': 32,
             'initial_table_id': 1},
            {'col_five': 'hehe',
             'col_four': False,
             'col_one': 'hello',
             'col_three': 90,
             'col_two': 22,
             'initial_table_id': 2}],
 'status': 'success',
 'version': 'dev'}


In [84]:
# Get view with where query
res = r.get(f'{url}/v3/pgrest/views/testview?where_col_two=22',
            headers=header_dat)
pp.pprint(res.json())

{'message': 'The request was successful.',
 'result': [{'col_five': 'hehe',
             'col_four': False,
             'col_one': 'hello',
             'col_three': 90,
             'col_two': 22,
             'initial_table_id': 2}],
 'status': 'success',
 'version': 'dev'}


In [85]:
# Delete view - Enter manage_view_id of the view
res = r.delete(f'{url}/v3/pgrest/manage/views/###',
               headers=header_dat)
pp.pprint(res.json())

{'message': 'View deleted successfully.',
 'result': None,
 'status': 'success',
 'version': 'dev'}


In [86]:
# Delete table - Enter table_id of the table
res = r.delete(f'{url}/v3/pgrest/manage/tables/###',
               headers=header_dat)
pp.pprint(res.json())

{'message': 'Table deleted successfully.',
 'result': None,
 'status': 'success',
 'version': 'dev'}
